In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import numpy as np
import gc

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50")
model = T5ForConditionalGeneration.from_pretrained("Rostlab/prot_t5_xl_uniref50")

d:\Documents\BROMBERG\CODE\code\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


NameError: name 'T5ForConditionalGeneration' is not defined

: 

In [9]:
gc.collect()

19

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [11]:
import sqlite3
# load the embedding data from the unshuffled sequences
# ids are in ruseq.txt
conn = sqlite3.connect('../cutoff/uniref90-10pm.db')
c = conn.cursor()

# get the embeddings
embeddings_unshuff = []
with open('../cutoff/rs/ruseq10000.txt') as f:
    keys = f.read().splitlines()

for key in keys:
    c.execute('SELECT embedding FROM uniref90 WHERE name = ?', (key,))
    embedding = c.fetchone()[0]
    if np.frombuffer(embedding, dtype=np.float16).shape[0] != 1024:
        embeddings_unshuff.append(np.frombuffer(embedding, dtype=np.float32))
    else:
        embeddings_unshuff.append(np.frombuffer(embedding, dtype=np.float16))
    break

conn.close()
print('loaded embeddings')
print(len(embeddings_unshuff))

loaded embeddings
1


In [15]:
def embedding_to_sequence(embedding):
    # Prepare the embedding tensor
    input_embedding = torch.tensor(embedding).unsqueeze(0).to(device)
    
    # Generate token IDs using model decoding
    with torch.no_grad():
        generated_ids = model.generate(input_embedding)
    
    # Decode token IDs into a sequence
    sequence = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return sequence

In [17]:
sequence = embedding_to_sequence(embeddings_unshuff[0])
print("Generated sequence from embedding:", sequence)

TypeError: The current model class (T5Model) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'T5ForConditionalGeneration'}